In [7]:
import numpy as np
import torch
import torch.nn.functional as F

P = torch.tensor([
    [.1,.4,.1,.2],
    [.3,.4,.5,.6],
    [.7,.3,.2,.1]
    ], dtype=torch.float)

Q = torch.tensor([
    [.3,.4,.2,.3],
    [.4,.5,.6,.1],
    [.1,.3,.4,.2]
    ], dtype=torch.float)

Y = torch.tensor([1,3,0])

phi_0 = - (P * torch.log(Q)).sum()
phi_1 = F.cross_entropy(P, Y, reduction='sum')

print(phi_0)
print(phi_1)


tensor(5.3950)
tensor(3.4753)


In [14]:
P = torch.tensor([
    [.1,.4,.1,.2],
    [.3,.4,.5,.6],
    [.7,.3,.2,.1]
    ], dtype=torch.float)

Q = torch.tensor([
    [.3,.4,.2,.3],
    [.4,.5,.6,.1],
    [.1,.3,.4,.2]
    ], dtype=torch.float)

Y = torch.tensor([1,3,0])

custome_crossentropy = lambda P, Q: - (P * torch.log(Q)).sum()

print(torch.nn.CrossEntropyLoss(reduction='sum')(P,Q))
print(torch.nn.functional.cross_entropy(P,Q, reduction='sum'))
print(custome_crossentropy(P,Q))

tensor(5.3896)
tensor(5.3896)
tensor(5.3950)
